In [47]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Fri Sep 23 08:06:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    41W / 250W |  12345MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [48]:
import numpy as np
import torch
import torch.nn as nn
import sys
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(43)

In [49]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Data pre-processing Steps**

In [50]:

def target_trans(target):
  y = -torch.ones(127)
  y[target] = 1

  global labels
  label = labels[target]
  for j in range(27):
    if label in get_descendants(labels[100+j]):
      y[100+j] = 1

  return y,target

In [51]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=[32,32]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = datasets.CIFAR100(root='data/', train=True, download=True, transform=transform_train,target_transform=target_trans)
test_dataset = datasets.CIFAR100(root='data/', train=False, download=True, transform=transform_test,target_transform=target_trans)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [52]:
#Tree Hierarchy of the labels

Tree = {"entity" : ["nature", "manmade"] , "nature":["flora", "bigAnimals", "smallAnimals", "aquaticAnimals", "scenes"], "manmade" : ["householdElectrical", "householdFurniture", "foodContainers", "vehicles1","vehicles2","largeManmadeOutdoorThings"],
        "flora": ["fruitsAndVegetables","flowers","trees"],"bigAnimals": ["largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals"],"smallAnimals":["smallMammals", "reptiles", "insects", "nonInsectInvertebrates"], "aquaticAnimals":["aquaticMammals", "fish"],"scenes":["cloud","forest","mountain","plain","sea"],
        "householdElectrical":["clock", "keyboard", "lamp", "telephone", "television"], "householdFurniture":["bed", "chair", "couch", "table", "wardrobe"],"foodContainers":["bottle", "bowl", "can", "cup", "plate"], "vehicles1":["bicycle", "bus", "motorcycle", "pickup_truck", "train"],"vehicles2":["lawn_mower", "rocket", "streetcar", "tank", "tractor"],"largeManmadeOutdoorThings":["bridge", "castle", "house", "road", "skyscraper"],
        "flowers":["orchid", "poppy", "rose", "sunflower", "tulip"],"fruitsAndVegetables":["apple", "mushroom", "orange", "pear", "sweet_pepper"],"trees":["maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree"],"largeCarnivores":["bear", "leopard", "lion", "tiger", "wolf"],"largeOmniAndHerbivores":["camel", "cattle", "chimpanzee", "elephant", "kangaroo"],"people":["baby", "boy", "girl", "man", "woman"],
        "mediumSizedMammals":["fox", "porcupine", "possum", "raccoon", "skunk"],"smallMammals":["hamster", "mouse", "rabbit", "shrew", "squirrel"],"reptiles":["crocodile", "dinosaur", "lizard", "snake", "turtle"],"insects":["bee", "beetle", "butterfly", "caterpillar", "cockroach"],"nonInsectInvertebrates":["crab", "lobster", "snail", "spider", "worm"],"aquaticMammals":["beaver", "dolphin", "otter", "seal", "whale"],
        "fish":["aquarium_fish", "flatfish", "ray", "shark", "trout"]}

labels = ["apple","aquarium_fish","baby","bear","beaver","bed","bee","beetle","bicycle","bottle","bowl","boy","bridge","bus","butterfly","camel","can","castle","caterpillar","cattle","chair","chimpanzee","clock","cloud","cockroach","couch","crab","crocodile","cup","dinosaur","dolphin","elephant","flatfish","forest","fox","girl","hamster","house","kangaroo","keyboard","lamp","lawn_mower","leopard","lion","lizard","lobster","man","maple_tree","motorcycle","mountain","mouse","mushroom","oak_tree","orange","orchid","otter","palm_tree","pear","pickup_truck","pine_tree","plain","plate","poppy","porcupine","possum","rabbit","raccoon","ray","road","rocket","rose","sea","seal","shark","shrew","skunk","skyscraper","snail","snake","spider","squirrel","streetcar","sunflower","sweet_pepper","table","tank","telephone","television","tiger","tractor","train","trout","tulip","turtle","wardrobe","whale","willow_tree","wolf","woman","worm", "flowers","fruitsAndVegetables", "trees", "largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals", "smallMammals", "reptiles", "insects", "nonInsectInvertebrates", "aquaticMammals", "fish", "flora", "bigAnimals", "smallAnimals","aquaticAnimals", "scenes", "householdElectrical", "householdFurniture", "foodContainers", "vehicles1", "vehicles2", "largeManmadeOutdoorThings", "nature" ,"manmade", "entity"]



In [53]:
#Level-wise representation of labels

level = [None]*5
level[0] = ["entity"]
level[1] = ["nature", "manmade"]
level[2] = ["flora", "bigAnimals", "smallAnimals", "aquaticAnimals", "scenes", "householdElectrical", "householdFurniture","foodContainers", "vehicles1", "vehicles2","largeManmadeOutdoorThings"]
level[3] = ["flowers", "fruitsAndVegetables", "trees", "largeCarnivores", "largeOmniAndHerbivores", "people", "medimSizedMammals", "smallMammals", "reptiles", "insects", "nonInsectInvertebrates", "aquaticMammals", "fish", "cloud", "forest", "mountain", "plain", "sea", "clock", "keyboard", "lamp", "telephone", "television", "bed", "chair", "couch", "table", "wardrobe", "bottle", "bowl", "can", "cup", "plate", "bicycle", "bus", "motorcycle", "pickup_truck", "train", "lawn_mower", "rocket", "streetcar", "tank", "tractor", "bridge", "castle", "house", "road", "skyscraper"]
level[4] = ["orchid", "poppy", "rose", "sunflower", "tulip", "apple", "mushroom", "orange", "pear", "sweet_pepper", "maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree", "bear", "leopard", "lion", "tiger", "wolf", "camel", "cattle", "chimpanzee", "elephant", "kangaroo", "baby", "boy", "girl", "man", "woman", "fox", "porcupine", "possum", "raccoon", "skunk", "hamster", "mouse", "rabbit", "shrew", "squirrel", "crocodile", "dinosaur", "lizard", "snake", "turtle", "bee", "beetle", "butterfly", "caterpillar", "cockroach", "crab", "lobster", "snail", "spider", "worm", "beaver", "dolphin", "otter", "seal", "whale", "aquarium_fish", "flatfish", "ray", "shark", "trout"]



# Input    - class label of the CIFAR100 dataset
# Function - To find the level of the given class label
# output   - Returns an integer which represents the level of the class label and returns -1 if it doesn't belong to any level
def get_level(node):
  if (node in level[0]):
    return 0
  elif (node in level[1]):
    return 1
  elif (node in level[2]):
    return 2
  elif (node in level[3]):
    return 3
  elif (node in level[4]):
    return 4
  else:
    return -1

# Input    - class label of the CIFAR100 dataset
# Function - To find all the children of the given class label using the Tree hierarchy
# output   - Returns an list of the labels which are children and returns None if the given label is a leaf

def get_children(node):
  if node in Tree.keys():
    return Tree[node]    
  else:
    return None


# Input    - class label of the CIFAR100 dataset
# Function - To find the parent of the given class label using the level and children function.
# output   - Returns the parent of the class label
def get_parent(node):
  l = get_level(node)
  if l == 0: 
    return node
  for i in level[l-1]:
    if(get_children(i) is not None):
      if node in get_children(i):
        return i
  return None

# Input    - class label of the CIFAR100 dataset and the level of the label
# Function - finds ancestors of the the given label. 
# output   - Returns the label itself if given level is greater than the label's level else returns the ancestor at level l
def get_ancestor(node, l):
  h = get_level(node)
  if l >= h:
    return node
  y = node
  for i in range(h-l):
    y = get_parent(y)
  return y

# Input    - class label of the CIFAR100 dataset
# Function - To find the descendants of the given class label including the given class label
# output   - Returns an list of class labels which are descendants else empty list if the class label is a leaf.
def get_descendants(node):
  c = get_children(node)
  d = []
  if c is not None:
    for i in c:
      d.extend(get_descendants(i)) 
    return d
  else :
    return [node] 


# Input    - two class labels of the CIFAR100 dataset
# Function - Finds the distance between the given two nodes
# output   - Returns an integer which represents the distance between class labels
def tree_loss(node1,node2):
  l1 = get_level(node1)
  l2 = get_level(node2)
  l = min(l1,l2)
  while l >= 0:
    if get_ancestor(node1,l) == get_ancestor(node2,l):
      break
    else :
      l = l-1
  return ((l1 + l2) - (2*l))


**OvA Cascade**

In [54]:
# Input    - output : Predicted output and target : True output of the input
# Function - For each batch of inputs, finds the tree loss using OvA algorithm.
# output   - Returns the floating integer by calculating the batch-wise tree loss

def tLoss(output,target):
  
  batch_size = target.size(0)
  num_classes = output.size(1)

  # Height of the hierarchy tree
  h = 4
  tau_values=[0,0,0,0,0]
  val = None
  pred = None
  loss = 0
  global labels

  # Iterate each example in the batch
  for i in range(batch_size):
    t = target[i]

    h = 4
    # searching int the bottom-up manner in the hierarchy tree
    while h >= 0:

      # Clone the output array to check at every level
      values = output.clone().detach()

      # Iterate over each class
      for j in range(num_classes):

        level = get_level(labels[j])

        # Values of nodes below the current level are considered in the ancestor hence we assign the min value
        if level > h:
          values[i,j] = -10000

        elif level < h:
          # Similary for the node which is above current level but is not a leaf node, we assign the min value
          if get_children(labels[j]) is not None:
            values[i,j] = -10000   

      val,pred = torch.max(values[i,:],0)

      # If the output is greater than the tau value at each level we consider the pred value assigned in the previous step
      if h == 4 and val >= tau_values[4]:
        break
      if h == 3 and val >= tau_values[3]:
        break
      if h == 2 and val >= tau_values[2]:
        break
      if h == 1 and val >= tau_values[1]:
        break
      if h == 0 and val >= tau_values[0]:
        break 
        
      h = h - 1
    
    loss = loss + tree_loss(labels[t],labels[pred])

  loss = loss/batch_size
  # Average loss for the batch is returned
  return loss


In [55]:
class Hinge_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Hinge_Loss,self).__init__()
        
    def forward(self,x,y):

        temp = (1 - (x * y))
        clamp = F.relu(temp)
        total_loss = torch.sum(clamp)/x.size(0)
        return total_loss

In [56]:
num_classes = 100
num_epochs = 100

In [57]:
class VGGForCiFar100(nn.Module):
  def __init__(self, hid_dim=5000, dropout=0.5, n_classes=100, use_fc=True, freeze=False):
    super().__init__()
    self.vgg = models.vgg16(pretrained=True)
    if not use_fc:
      self.vgg.classifier = nn.Linear(512 * 7 * 7, n_classes)
    else:
      self.vgg.classifier = nn.Sequential(
              nn.Linear(512 * 7 * 7, 5000),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              #nn.Linear(4096, 4096),
              #nn.ReLU(True),
              #nn.Dropout(p=dropout),
              nn.Linear(5000, n_classes),
          )
      
    if freeze:
      for param in self.vgg.features.parameters():
        param.requires_grad = False
      for param in self.vgg.avg_pool.parameters():
        param.requires_grad = False
      for param in self.vgg.flatten.parameters():
        param.requires_grad = False
    

  def forward(self, x):
    return self.vgg(x)

In [58]:
model = VGGForCiFar100(n_classes=127).to(device)
criterion = Hinge_Loss().to(device)
accuracy = Accuracy(num_classes=num_classes).to(device)
optimizer = Ranger(model.parameters(), lr=1e-3,weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)


In [59]:
model

VGGForCiFar100(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [60]:
def get_accuracy(output, target):
  batch_size = target.size(0)

  pred = output[:,0:100]      # considering the first 100 of the 127 nodes because they are the true classes(No internal node)
  pred = pred.max(dim=1)[1]

  correct = pred==target
  acc = correct.float().sum(0)

  return acc/batch_size

**Training and Validation**

In [61]:
#Training

total_train_step = len(train_loader)
#print(total_train_step)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None


for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        y_trans = target[0]
        y_true = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        train_acc += get_accuracy(outputs, y_true)
        #train_acc += accuracy(outputs[:,0:100], y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # scheduler.step(train_loss/total_train_step)
        

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        Tree_Loss_Value = 0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_trans = target[0]
            y_true = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, y_trans)
            Tree_Loss_Value += tLoss(outputs,y_true)
            val_acc += get_accuracy(outputs, y_true)
            #val_acc += accuracy(outputs[:,0:100], y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 

    print(f'Accuracy of the network on validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}, Tree loss: {(Tree_Loss_Value/total_val_step):.4f}') 

Epoch [1/100] - Loss: 17.9507, Accuracy: 0.0665
Accuracy of the network on validation images: 0.2272, loss: 5.8182, Tree loss: 2.7839
Epoch [2/100] - Loss: 5.5816, Accuracy: 0.1822
Accuracy of the network on validation images: 0.3281, loss: 4.9392, Tree loss: 2.4604
Epoch [3/100] - Loss: 4.8552, Accuracy: 0.2918
Accuracy of the network on validation images: 0.4001, loss: 4.4262, Tree loss: 2.2474
Epoch [4/100] - Loss: 4.3643, Accuracy: 0.3735
Accuracy of the network on validation images: 0.4446, loss: 4.1038, Tree loss: 2.0890
Epoch [5/100] - Loss: 4.0092, Accuracy: 0.4315
Accuracy of the network on validation images: 0.4796, loss: 3.8846, Tree loss: 1.9870
Epoch [6/100] - Loss: 3.7283, Accuracy: 0.4717
Accuracy of the network on validation images: 0.5000, loss: 3.6859, Tree loss: 1.8753
Epoch [7/100] - Loss: 3.4842, Accuracy: 0.5116
Accuracy of the network on validation images: 0.5283, loss: 3.5871, Tree loss: 1.8166
Epoch [8/100] - Loss: 3.2350, Accuracy: 0.5415
Accuracy of the netwo

**Testing**

In [62]:

#Testing
model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0
    Tree_Loss_Value=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_trans = target[0]
        y_true = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        Tree_Loss_Value += tLoss(outputs,y_true)
        test_acc += get_accuracy(outputs, y_true)
        #test_acc += accuracy(outputs[:,0:100], y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}, Tree loss: {(Tree_Loss_Value/total_test_step):.4f}')

Accuracy of the network on test images: 0.6692, loss: 3.1885, Tree loss: 1.3100
